In [23]:
import json
import os
import random
import numpy as np
import scipy.stats as stats

dataset = 'cifar10'
# dataset_dir = f'../../Datasets/{dataset}'
dataset_dir = f'dataset/{dataset}'
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'test')

In [26]:
repeats = [1]
sigmas = [0.1]
seeds = [1, 2, 3]
init_cls = 1.0

In [8]:
for repeat in repeats:
    for sigma in sigmas:
        for seed in seeds:
            random.seed(seed)
            np.random.seed(seed)
            cls_list = os.listdir(train_dir)
            n_classes = len(cls_list)
            random.shuffle(cls_list)
            cls_dict = {cls:i for i, cls in enumerate(cls_list)}
            n_cls_seen = np.round(n_classes*(init_cls + (1-init_cls)*(np.arange(repeat)+1)/repeat)).astype(np.int)
            cls_order_list = np.arange(n_classes)
            np.random.shuffle(cls_order_list)
            cls_increment_time = np.zeros(n_classes)
            for i in range(repeat):
                if i > 0:
                    cls_increment_time[cls_order_list[n_cls_seen[i-1]:n_cls_seen[i]]] = i
            samples_list = []
            for cls in cls_list:
                cls_sample_list = os.listdir(os.path.join(train_dir, cls))
                random.shuffle(cls_sample_list)
                samples_list.append(cls_sample_list)
            stream = []
            for i in range(n_classes):
                times = np.random.normal(i/n_classes, sigma, size=len(samples_list[i]))
                choice = np.random.choice(repeat, size=len(samples_list[i]))
                times += choice
                for ii, sample in enumerate(samples_list[i]):
                    if choice[ii] >= cls_increment_time[i]:
                        stream.append({'file_name': os.path.join('train/'+cls_list[i], samples_list[i][ii]), 'klass': cls_list[i], 'label':i, 'time':times[ii]})
            random.shuffle(stream)
            stream = sorted(stream, key=lambda d: d['time'])
            data = {'cls_dict':cls_dict, 'stream':stream, 'cls_addition':list(cls_increment_time)}

            with open(f'collections/{dataset}/{dataset}_sigma{int(sigma*100)}_repeat{repeat}_init{int(init_cls*100)}_seed{seed}.json', 'w') as fp:
                json.dump(data, fp)

In [16]:
val = []
cls_list = os.listdir(train_dir)
n_classes = len(cls_list)
cls_dict = {cls:i for i, cls in enumerate(cls_list)}
for i in range(n_classes):
    cls_val_list = os.listdir(os.path.join(val_dir, cls_list[i]))
    for ii, sample in enumerate(cls_val_list):
        val.append({'file_name': os.path.join('val/', cls_val_list[ii]), 'klass': cls_list[i], 'label':i})

with open(f'collections/{dataset}/{dataset}_val2.json', 'w') as fp:
    json.dump(val, fp)